In [117]:
from gurobipy import Model, GRB, quicksum
from data_retrieval import DataGenerationModule
import pandas as pd
from math import sin, radians, cos, sqrt, atan2
import pickle
import numpy as np

pd.options.mode.chained_assignment = None

In [118]:
# Initialize the model
model = Model("Courier_Assignment")

In [119]:
# File paths
courier_wave_path = "courier_wave_info_meituan.csv"
all_waybill_path = "all_waybill_info_meituan_0322.csv"
dispatching_order_path = "dispatch_rider_meituan.csv"
dispatch_waybill_path = "dispatch_waybill_meituan.csv"

# Initialize the data generation module
data_module = DataGenerationModule(
    courier_wave_path,
    all_waybill_path,
    dispatching_order_path,
    dispatch_waybill_path,
)

/Users/youssef/Documents/Fall-2024/Practical-DS/Project/data_retrieval.py:147: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  order["active_area_orders"].fillna(0, inplace=True)


In [120]:
def haversine_distance(lat1, lon1, lat2, lon2):
    # Convert from microdegrees to degrees
    lat1, lon1, lat2, lon2 = [x / 1e6 for x in [lat1, lon1, lat2, lon2]]
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    radius = 6371.0
    return radius * c

In [121]:
# Initialize simulation parameters
num_periods = 6  # Total time periods to simulate
period_length = 60  # in seconds
start = 1665936258
average_courier_speed = 40

# Generate data for each period
all_orders = {}  # Dictionary to store orders per period
all_couriers = {}  # Dictionary to store couriers per period
order_location = {}  # Locations of all orders
courier_location = {}  # Start locations of couriers
prep_time = {}  # Preparation times for all orders
courier_arrival_time = {}  # Arrival times for couriers
a = {}  # Acceptance parameter

In [122]:
# Decision variables (dictionary to hold variables for multiple periods)
x = {}

# Cumulative objectives
avg_waiting_time = []
avg_distance = []
avg_workload_imbalance = []

In [123]:
def construct_orders_dict(
    orders: pd.DataFrame,
    period: int,
    all_orders: dict,
    prep_time: dict,
    order_location: dict,
):
    all_orders[period] = list(set(orders["order_id"]))
    for i in all_orders[period]:
        time = orders[orders["order_id"] == i]["estimate_meal_prepare_time"].iloc[-1]
        if time == 0:
            time = orders[orders["order_id"] == i]["order_push_time"].iloc[-1]
        prep_time[i] = time

        location = (
            orders[orders["order_id"] == i]["sender_lat"].iloc[-1],
            orders[orders["order_id"] == i]["sender_lng"].iloc[-1],
        )
        order_location[i] = location

In [124]:
def construct_couriers_dict(
    couriers: pd.DataFrame,
    period: int,
    all_couriers: dict,
    courier_location: dict,
):
    all_couriers[period] = list(set(couriers["courier_id"]))
    for i in all_couriers[period]:
        location = (
            couriers[couriers["courier_id"] == i]["grab_lat"].iloc[-1],
            couriers[couriers["courier_id"] == i]["grab_lng"].iloc[-1],
        )
        courier_location[i] = location

In [125]:
def construct_arrival_time_dict(
    period: int,
    all_orders: dict,
    all_couriers: dict,
    order_location: dict,
    courier_location: dict,
    courier_arrival_time: dict,
    current_time: int,
):
    for i in all_orders[period]:
        for j in all_couriers[period]:

            distance = haversine_distance(
                order_location[i][0],
                order_location[i][1],
                courier_location[j][0],
                courier_location[j][1],
            )

            arrival_time = int(distance / (average_courier_speed / 3600)) + current_time
            courier_arrival_time[i, j, period] = arrival_time

In [126]:
def construct_acceptance_dict(
    period: int, all_orders: dict, all_couriers: dict, a: dict
):
    for i in all_orders[period]:
        for j in all_couriers[period]:
            features = {
                "active_orders": data_module.all_waybill_df[
                    data_module.all_waybill_df["courier_id"] == j
                ]["active_orders"].iloc[-1],
                "hour_of_day": data_module.all_waybill_df[
                    data_module.all_waybill_df["order_id"] == i
                ]["hour_of_day"].iloc[-1],
                "is_weekend": data_module.all_waybill_df[
                    data_module.all_waybill_df["order_id"] == i
                ]["is_weekend"].iloc[-1],
                "historical_rejection_rate": data_module.all_waybill_df[
                    data_module.all_waybill_df["courier_id"] == j
                ]["historical_rejection_rate"].iloc[-1],
                "da_id": data_module.all_waybill_df[
                    data_module.all_waybill_df["order_id"] == i
                ]["da_id"].iloc[-1],
                "peak_hours": data_module.all_waybill_df[
                    data_module.all_waybill_df["courier_id"] == j
                ]["peak_hours"].iloc[-1],
                "active_area_orders": data_module.all_waybill_df[
                    data_module.all_waybill_df["order_id"] == i
                ]["active_area_orders"].iloc[-1],
                "near_shift_end": data_module.all_waybill_df[
                    data_module.all_waybill_df["courier_id"] == j
                ]["near_shift_end"].iloc[-1],
            }

            features_df = pd.DataFrame([features])

            with open("logreg_model.pkl", "rb") as f:
                logreg_model = pickle.load(f)

            prediction = logreg_model.predict(features_df)

            a[i, j, period] = prediction

In [127]:
# Loop over each time period
for t in range(1, num_periods + 1):
    print(f"\n===== Time Period {t} =====")
    # Generate orders
    orders_this_period = data_module.get_orders_in_time_window(
        start_time=start + (t - 1) * period_length, end_time=start + (t) * period_length
    )
    construct_orders_dict(
        orders=orders_this_period,
        period=t,
        all_orders=all_orders,
        prep_time=prep_time,
        order_location=order_location,
    )

    couriers_this_period = data_module.get_active_couriers(
        timestamp=start + (t - 1) * period_length
    )
    construct_couriers_dict(
        couriers=couriers_this_period,
        period=t,
        all_couriers=all_couriers,
        courier_location=courier_location,
    )

    construct_arrival_time_dict(
        period=t,
        all_orders=all_orders,
        all_couriers=all_couriers,
        order_location=order_location,
        courier_location=courier_location,
        courier_arrival_time=courier_arrival_time,
        current_time=start + (t - 1) * period_length,
    )

    construct_acceptance_dict(
        period=t, all_orders=all_orders, all_couriers=all_couriers, a=a
    )

    # Add new decision variables for this period
    for i in all_orders[t]:
        for j in all_couriers[t]:
            x[i, j, t] = model.addVar(vtype=GRB.BINARY, name=f"x_{i}_{j}_{t}")

    # Update workload
    workload = {
        j: quicksum(x[i, j, t] for i in all_orders[t] if (i, j, t) in x)
        for j in all_couriers[t]
    }
    mean_workload = quicksum(workload[j] for j in workload) / len(workload)

    # Auxiliary variables for workload imbalance
    y = model.addVars(workload.keys(), lb=0, name=f"y_t{t}")

    # Objectives
    waiting_time_objective = quicksum(
        abs(prep_time[i] - courier_arrival_time[i, j, t]) * x[i, j, t]
        for i in all_orders[t]
        for j in all_couriers[t]
    ) / len(all_orders[t])
    distance_objective = quicksum(
        haversine_distance(
            order_location[i][0],
            order_location[i][1],
            courier_location[j][0],
            courier_location[j][1],
        )
        * x[i, j, t]
        for i in all_orders[t]
        for j in all_couriers[t]
    ) / len(all_orders[t])

    workload_imbalance_objective = quicksum(y[j] for j in workload) / len(
        all_couriers[t]
    )

    # Set Objectives
    model.setObjectiveN(
        waiting_time_objective, index=0, priority=1, name=f"MinimizeWaitingTime_t{t}"
    )
    model.setObjectiveN(
        distance_objective, index=1, priority=1, name=f"MinimizeDistance_t{t}"
    )
    model.setObjectiveN(
        workload_imbalance_objective,
        index=2,
        priority=1,
        name=f"MinimizeWorkloadImbalance_t{t}",
    )

    # Constraints
    for i in all_orders[t]:
        model.addConstr(
            quicksum(x[i, j, t] for j in all_couriers[t]) == 1, name=f"assign_{i}_t{t}"
        )
    for i in all_orders[t]:
        for j in all_couriers[t]:
            model.addConstr(x[i, j, t] <= a[i, j, t], name=f"acceptance_{i}_{j}_t{t}")

    for j in workload:
        model.addConstr(
            y[j] >= workload[j] - mean_workload, name=f"abs_dev_pos_{j}_t{t}"
        )
        model.addConstr(
            y[j] >= mean_workload - workload[j], name=f"abs_dev_neg_{j}_t{t}"
        )

    # Optimize the model for this period
    model.optimize()

    # Get current period's objectives
    waiting_time_value = waiting_time_objective.getValue()
    distance_value = distance_objective.getValue()
    workload_imbalance_value = workload_imbalance_objective.getValue()

    # Update cumulative objectives
    avg_waiting_time.append(waiting_time_value)
    avg_distance.append(distance_value)
    avg_workload_imbalance.append(workload_imbalance_value)

    # Print the results for this time period
    print("\nObjective Values for Current Period:")
    print(f"Waiting Time: {waiting_time_value}")
    print(f"Distance Traveled: {distance_value}")
    print(f"Workload Imbalance: {workload_imbalance_value}")

    print("\nAssignments:")
    for i in all_orders[t]:
        for j in all_couriers[t]:
            if x[i, j, t].x > 0.5:  # Check if variable is assigned
                data_module.all_waybill_df[
                    data_module.all_waybill_df["courier_id"] == j
                ]["active_orders"] += 1
                print(f"Order {i} is assigned to Courier {j} in Time Period {t}")


===== Time Period 1 =====
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.1.0 24B91)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 18 rows, 12 columns and 88 nonzeros
Model fingerprint: 0xe455ff38
Variable types: 4 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  Objective range  [2e-01, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 3 objectives (1 combined)...
---------------------------------------------------------------------------
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (weighted) ...
---------------------------------------------------------------------------

Optimize a model with 18 rows, 12 columns and 88 nonz

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentV

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.1.0 24B91)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 112 rows, 87 columns and 938 nonzeros
Model fingerprint: 0x7eea3432
Variable types: 9 continuous, 78 integer (78 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  Objective range  [2e-02, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 3 objectives (1 combined)...
---------------------------------------------------------------------------
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (weighted) ...
---------------------------------------------------------------------------

Optimize a model with 112 rows, 87 columns and 938 nonzeros
Model fingerprin

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentV


Objective Values for Current Period:
Waiting Time: 533.7142857142858
Distance Traveled: 8.298057031512775
Workload Imbalance: 1.7599999999999996

Assignments:
Order 0 is assigned to Courier 247 in Time Period 2
Order 321188 is assigned to Courier 1125 in Time Period 2
Order 301605 is assigned to Courier 247 in Time Period 2
Order 290312 is assigned to Courier 1125 in Time Period 2
Order 329384 is assigned to Courier 1125 in Time Period 2
Order 238858 is assigned to Courier 1125 in Time Period 2
Order 270442 is assigned to Courier 3447 in Time Period 2
Order 197747 is assigned to Courier 1125 in Time Period 2
Order 340437 is assigned to Courier 864 in Time Period 2
Order 206614 is assigned to Courier 1125 in Time Period 2
Order 230134 is assigned to Courier 864 in Time Period 2
Order 203674 is assigned to Courier 1125 in Time Period 2
Order 85916 is assigned to Courier 864 in Time Period 2
Order 259070 is assigned to Courier 3447 in Time Period 2

===== Time Period 3 =====


...

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentV


Objective Values for Current Period:
Waiting Time: 256.66666666666663
Distance Traveled: 4.713473800570307
Workload Imbalance: 0.47086801426872293

Assignments:
Order 76481 is assigned to Courier 1065 in Time Period 5
Order 454213 is assigned to Courier 3638 in Time Period 5
Order 264775 is assigned to Courier 2274 in Time Period 5
Order 215625 is assigned to Courier 368 in Time Period 5
Order 351529 is assigned to Courier 2274 in Time Period 5
Order 482994 is assigned to Courier 1065 in Time Period 5
Order 503829 is assigned to Courier 1125 in Time Period 5
Order 230135 is assigned to Courier 261 in Time Period 5
Order 60795 is assigned to Courier 1860 in Time Period 5

===== Time Period 6 =====


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentV

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.1.0 24B91)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1349 rows, 1184 columns and 55332 nonzeros
Model fingerprint: 0xd7661b63
Variable types: 104 continuous, 1080 integer (1080 binary)
Coefficient statistics:
  Matrix range     [3e-02, 1e+00]
  Objective range  [1e-02, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 3 objectives (1 combined)...
---------------------------------------------------------------------------
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (weighted) ...
---------------------------------------------------------------------------

Optimize a model with 1349 rows, 1184 columns and 55332 nonzeros


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentV

In [128]:
print("\n===== Final Average Results =====")
print(f"Avg Waiting Time: {np.mean(avg_waiting_time)}")
print(f"Avg Distance Traveled: {np.mean(avg_distance)}")
print(f"Avg Workload Imbalance: {np.mean(avg_workload_imbalance)}")


===== Final Average Results =====
Avg Waiting Time: 340.8588624338625
Avg Distance Traveled: 5.965796031507236
Avg Workload Imbalance: 0.8620330038322656
